In [ ]:
import warnings
warnings.simplefilter("ignore")
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
sys.path.append('/home/ubuntu/Mask_RCNN/samples/coco') 
import coco
%matplotlib inline

In [ ]:
# 2 class - clown and others (labelImg)
epochs = 1
STEPS_PER_EPOCH = 17



class ClownDataset(Dataset):
	def load_dataset(self, dataset_dir, is_train=True):
		self.add_class("dataset", 1, "clown")
		self.add_class("dataset", 2, "others")
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		for filename in listdir(images_dir):
			image_id = filename[:-4]
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

	def extract_boxes(self, filename):
		tree = ElementTree.parse(filename)
		root = tree.getroot()
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		info = self.image_info[image_id]
		path = info['annotation']
		boxes, w, h = self.extract_boxes(path)
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if i == 0:                                    
				masks[row_s:row_e, col_s:col_e, i] = 1                 #mod  
				class_ids.append(self.class_names.index('clown'))      #mod
			else:                                         
				masks[row_s:row_e, col_s:col_e, i] = 2                 #mod
				class_ids.append(self.class_names.index('others'))     #mod
		return masks, asarray(class_ids, dtype='int32')                #mod

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

class ClownConfig(Config):
	NAME = "Clown_cfg"
	NUM_CLASSES = 1 + 2
	STEPS_PER_EPOCH = STEPS_PER_EPOCH

train_set = ClownDataset()
train_set.load_dataset('/home/ubuntu/Mask_RCNN/data/training', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
test_set = ClownDataset()
test_set.load_dataset('/home/ubuntu/Mask_RCNN/data/validation', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
config = ClownConfig()
config.display()

model = MaskRCNN(mode='training', model_dir='./', config=config)
model.load_weights('/home/ubuntu/Mask_RCNN/models/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=epochs, layers='heads')